In [ ]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal,Annotated
from langchagentn_openagent import ChatOpenagent
from langchagentn_core.messages import BaseMessage,HumanMessage
from pydantic import BaseModel,Field
from langgraph.graph.message import  add_messages


In [7]:
class ChatGraphState(TypedDict):
    old_conversation:dict
    latest_chat:str



In [9]:

  messages= [
    {
      "role": "user",
      "content": "Hi, my order hasn’t arrived yet. Can you help?"
    },
    {
      "role": "agent",
      "content": "Of course! Could you please provide your order number so I can check the status?"
    },
    {
      "role": "user",
      "content": "Yes, it’s #ORD7821."
    },
    {
      "role": "agent",
      "content": "Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM."
    },
    {
      "role": "user",
      "content": "Great, thanks for the update!"
    },
    {
      "role": "agent",
      "content": "You’re welcome! Is there anything else I can help you with today?"
    },
    {
      "role": "user",
      "content": "No, that’s all. Have a nice day!"
    },
    {
      "role": "agent",
      "content": "You too! 😊"
    }
  ]


In [ ]:
tool=[
    {
        "type": "function",
        "function": {
            "name": "search_database",
            "description": "generate reply based on the latest chat",
            "parameters": {
                "type": "object",
                "properties": {
                    "old_conversation": {
                        "type": "dict",
                        "description": "the old conversation"
                    },
                    "latest_chat": {
                        "type": "string",
                        "description": "the latest chat"
                    }
                },
                "required": [
                    "latest_chat"
                ]
            }
        }
    }
]

In [ ]:
def search_database(query:str):
    return {"results": f"Results for '{query}'"}

In [ ]:
def gen_reply(state:ChatGraphState):
    print(state)
    prompt=f""" 
    you are the customer support Agent who gracefully repond the user query from the provied search tool only. 
    """
    llm=ChatOpenagent(model="gpt-3.5-turbo-1106")
    response=llm.invoke([HumanMessage(content=state['latest_chat'])])
    return 
    # return {'old_conversation':add_messages(state['old_conversation'],response),'latest_chat':response.content})

SyntaxError: unmatched ')' (2704427689.py, line 5)

In [8]:
graph= StateGraph(ChatGraphState)
graph.add_node('gen_reply',gen_reply)
graph.add_edge(START,'gen_reply')
graph.add_edge('gen_reply',END)

NameError: name 'gen_reply' is not defined

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# ----- STATE -----
class ChatGraphState(TypedDict):
    old_conversation: list[BaseMessage]
    latest_chat: str

# ----- TOOL SCHEMA -----
class SearchDatabaseInput(BaseModel):
    old_conversation: list[BaseMessage] = Field(..., description="The old conversation")
    latest_chat: str = Field(..., description="The latest user message")

def search_database(old_conversation: list[BaseMessage], latest_chat: str):
    # Convert messages to text
    conversation_text = "\n".join([msg.content for msg in old_conversation])
    return {"results": f"Based on previous conversation:\n{conversation_text}\nAnswer to: {latest_chat}"}

# ----- SAMPLE CONVERSATION -----
messages = [
    {"role": "user", "content": "Hi, my order hasn’t arrived yet. Can you help?"},
    {"role": "agent", "content": "Of course! Could you please provide your order number so I can check the status?"},
    {"role": "user", "content": "Yes, it’s #ORD7821."},
    {"role": "agent", "content": "Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM."},
    {"role": "user", "content": "Great, thanks for the update!"},
    {"role": "agent", "content": "You’re welcome! Is there anything else I can help you with today?"},
    {"role": "user", "content": "No, that’s all. Have a nice day!"},
    {"role": "agent", "content": "You too! 😊"}
]

# Convert dict messages to BaseMessage objects
converted_messages = []
for msg in messages:
    if msg["role"] == "user":
        converted_messages.append(HumanMessage(content=msg["content"]))
    else:
        converted_messages.append(AIMessage(content=msg["content"]))

# ----- NODE -----
def gen_reply(state: ChatGraphState):
    print("STATE:", state)

    llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
    llm_with_tools = llm.bind_tools([search_database], tool_choice="auto")

    # Invoke LLM
    response = llm_with_tools.invoke([HumanMessage(content=state["latest_chat"])])

    # If tool was called
    if hasattr(response, "tool_calls") and response.tool_calls:
        for call in response.tool_calls:
            if call["name"] == "search_database":
                tool_result = search_database(state["old_conversation"], state["latest_chat"])
                response = AIMessage(content=tool_result["results"])

    # Add the LLM response to the conversation
    return {
        # "old_conversation": add_messages(state["old_conversation"], [response]),
        "latest_chat": state["latest_chat"],
    }

# ----- GRAPH -----
graph = StateGraph(ChatGraphState)
graph.add_node("gen_reply", gen_reply)
graph.add_edge(START, "gen_reply")
graph.add_edge("gen_reply", END)

chatbot = graph.compile()

# ----- INITIAL STATE -----
initial_state = {
    "old_conversation": converted_messages,
    "latest_chat": "Hello, I forgot my password?"
}

# Run chatbot
result = chatbot.invoke(initial_state)
print("FINAL RESULT:", result)


STATE: {'old_conversation': [HumanMessage(content='Hi, my order hasn’t arrived yet. Can you help?', additional_kwargs={}, response_metadata={}), AIMessage(content='Of course! Could you please provide your order number so I can check the status?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, it’s #ORD7821.', additional_kwargs={}, response_metadata={}), AIMessage(content='Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Great, thanks for the update!', additional_kwargs={}, response_metadata={}), AIMessage(content='You’re welcome! Is there anything else I can help you with today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='No, that’s all. Have a nice day!', additional_kwargs={}, response_metadata={}), AIMessage(content='You too! 😊', additional_kwargs={}, response_metadata={})], 'latest_chat': 'Hello, I forgot my password?'

In [21]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# ----- STATE -----
class ChatGraphState(TypedDict):
    old_conversation: list[BaseMessage]
    latest_chat: str
    tool_calls: list[dict]  # to store tool call details

# ----- TOOL SCHEMA -----
class SearchDatabaseInput(BaseModel):
    old_conversation: list[BaseMessage] = Field(..., description="The old conversation")
    latest_chat: str = Field(..., description="The latest user message")

def search_database(old_conversation: list[BaseMessage], latest_chat: str):
    # Convert messages to text
    conversation_text = "\n".join([msg.content for msg in old_conversation])
    return {"results": f"Based on previous conversation:\n{conversation_text}\nAnswer to: {latest_chat}"}

# ----- SAMPLE CONVERSATION -----
messages = [
    {"role": "user", "content": "Hi, my order hasn’t arrived yet. Can you help?"},
    {"role": "agent", "content": "Of course! Could you please provide your order number so I can check the status?"},
    {"role": "user", "content": "Yes, it’s #ORD7821."},
    {"role": "agent", "content": "Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM."},
    {"role": "user", "content": "Great, thanks for the update!"},
    {"role": "agent", "content": "You’re welcome! Is there anything else I can help you with today?"},
    {"role": "user", "content": "No, that’s all. Have a nice day!"},
    {"role": "agent", "content": "You too! 😊"}
]

# Convert dict messages to BaseMessage objects
converted_messages = []
for msg in messages:
    if msg["role"] == "user":
        converted_messages.append(HumanMessage(content=msg["content"]))
    else:
        converted_messages.append(AIMessage(content=msg["content"]))

# ----- NODE -----
def gen_reply(state: ChatGraphState):
    print("STATE:", state)

    llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
    llm_with_tools = llm.bind_tools([search_database], tool_choice="auto")

    # Invoke LLM
    response = llm_with_tools.invoke([HumanMessage(content=state["latest_chat"])])

    tool_call_details = []

    # If tool was called
    if hasattr(response, "tool_calls") and response.tool_calls:
        for call in response.tool_calls:
            if call["name"] == "search_database":
                args = {"old_conversation": state["old_conversation"], "latest_chat": state["latest_chat"]}
                result = search_database(**args)
                tool_call_details.append({"tool_name": call["name"], "args": args, "result": result})

                # Wrap tool result as AIMessage
                response = AIMessage(content=result["results"])

    # Add the LLM response to the conversation
    return {
        "old_conversation": add_messages(state["old_conversation"], [response]),
        "latest_chat": state["latest_chat"],
        "tool_calls": tool_call_details
    }

# ----- GRAPH -----
graph = StateGraph(ChatGraphState)
graph.add_node("gen_reply", gen_reply)
graph.add_edge(START, "gen_reply")
graph.add_edge("gen_reply", END)

chatbot = graph.compile()

# ----- INITIAL STATE -----
initial_state = {
    "old_conversation": converted_messages,
    "latest_chat": "Hello, I forgot my password?",
    "tool_calls": []
}

# Run chatbot
result = chatbot.invoke(initial_state)
print("FINAL RESULT:", result)


STATE: {'old_conversation': [HumanMessage(content='Hi, my order hasn’t arrived yet. Can you help?', additional_kwargs={}, response_metadata={}), AIMessage(content='Of course! Could you please provide your order number so I can check the status?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, it’s #ORD7821.', additional_kwargs={}, response_metadata={}), AIMessage(content='Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Great, thanks for the update!', additional_kwargs={}, response_metadata={}), AIMessage(content='You’re welcome! Is there anything else I can help you with today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='No, that’s all. Have a nice day!', additional_kwargs={}, response_metadata={}), AIMessage(content='You too! 😊', additional_kwargs={}, response_metadata={})], 'latest_chat': 'Hello, I forgot my password?'

In [23]:
print("FINAL RESULT:", result["tool_calls"])


FINAL RESULT: [{'tool_name': 'search_database', 'args': {'old_conversation': [HumanMessage(content='Hi, my order hasn’t arrived yet. Can you help?', additional_kwargs={}, response_metadata={}, id='979a2f5e-45d7-4da5-8419-247b7f88e717'), AIMessage(content='Of course! Could you please provide your order number so I can check the status?', additional_kwargs={}, response_metadata={}, id='88f91c60-3f04-4cd8-a48a-0cc917f95d61'), HumanMessage(content='Yes, it’s #ORD7821.', additional_kwargs={}, response_metadata={}, id='6922c103-e69f-4699-8d10-523d331c72c6'), AIMessage(content='Thanks! I’ve checked the system and your order is on the way. It should arrive tomorrow by 8 PM.', additional_kwargs={}, response_metadata={}, id='2dd6956f-668e-450d-af94-afc9377f68cf'), HumanMessage(content='Great, thanks for the update!', additional_kwargs={}, response_metadata={}, id='005a74b9-e280-45bd-bdd2-9c48c5214fbd'), AIMessage(content='You’re welcome! Is there anything else I can help you with today?', additi